# 05 CNN Forward Method - PyTorch Deep Learning Implementation
## Review

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

The `forward()` method is the **actual network transformation**. <br>The forward method is the mapping that maps an **input tensor** to a prediction **output tensor**. Let's see how this is done.

This means that the forward method implementation will use **all of the layers** we defined inside the constructor. In this way, the **forward method** explicitly defines the **network's transformation**.

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size = 5)
        
        self.fc1 = nn.Linear(in_features = 12*4*4,out_features = 120)
        self.fc2 = nn.Linear(in_features = 120,out_features = 60)
        self.out = nn.Linear(in_features = 60,out_features = 10)
        
    def forward(self,t):
        # implement the forward pass
        return t

## Input Layer 1

The **input layer** of any neural network is determined by the **input data**. 

```python
def forward(self,t):
    #(1) input ;ayer
    t = t
```

## Hidden Convolutional Layers: Layers 2 And 3

To preform the convolution operation, we pass the tensor to the forward method of the first convolutional layer, `self.conv1`. We've learned how all PyTorch neural network modules have `forward()` methods, and when we call the `forward()` method of a `nn.Module`, there is a special way that we make the call.

When want to call the `forward()` method of a `nn.Module` instance, we call the actual instance instead of calling the `forward()` method directly.

Instead of doing this `self.conv1.forward(tensor)`, we do this `self.conv1(tensor)`.

```python
# (2) hidden conv layer
t = self.conv1(t)
t = F.relu(t)
t = F.max_pool2d(t, kernel_size=2, stride=2)

# (3) hidden conv layer
t = self.conv2(t)
t = F.relu(t)
t = F.max_pool2d(t, kernel_size=2, stride=2)
```

As we can see here, our input tensor is transformed as we move through the convolutional layers. The first convolutional layer has a convolutional operation, followed by a **relu activation** operation whose output is then passed to a max pooling operation with `kernel_size=2` and `stride=2`.(其中kernel size就是filter的大小)

The output tensor `t` of the first convolutional layer is then passed to the next convolutional layer, which is identical except for the fact that we call `self.conv2()` instead of `self.conv1()`.

Each of these layers is comprised of a collection of weights (data) and a collection operations (code). The weights are encapsulated inside the `nn.Conv2d()` class instance. The `relu()` and the `max_pool2d()` calls are just **pure operations**. Neither of these have weights, and this is why we call them directly from the `nn.functional` API.

Sometimes we may see `pooling operations` referred to as pooling layers. Sometimes we may even hear activation operations called activation layers.

**However**, what makes a layer **distinct** from an operation is that layers have **weights**. Since `pooling` operations and `activation` functions **do not have weights**, we will refer to them as **operations** and view them as being added to the collection of layer operations.

**确定一个操作能不能叫做layer就看他有没有weights，不过也别被这些术语搞糊涂了，我们就是通过一系列方法的组合来实现这个`forward()`**

Don't let these terms confuse the fact that the whole network is simply a composition of functions, and what we are doing now is defining this composition inside our `forward()` method.

## Hidden Linear Layers: Layers 4 And 5
Before we pass our input to the first hidden linear layer, we must `reshape()` or `flatten` our tensor. This will be the case any time we are passing output from a convolutional layer as input to a linear layer.

Since the forth layer is the first linear layer, we will include our reshaping operation as a part of the forth layer.

```python
# (4) hidden linear layer
t = t.reshape(-1, 12 * 4 * 4)
t = self.fc1(t)
t = F.relu(t)

# (5) hidden linear layer
t = self.fc2(t)
t = F.relu(t)
```

number `12` in the reshaping operation is determined by the number of output channels coming from the previous convolutional layer(`out_channels=12`).

The `4 * 4` is actually the height and width of each of the 12 output channels.

The height and width dimensions have been reduced from `28 x 28` to `4 x 4` by the convolution and pooling operations.

After the tensor is reshaped, we pass the `flattened` tensor to the linear layer and pass this result to the `relu() activation function`.

## Output Layer 6

The sixth and last layer of our network is a linear layer we call the **output layer**. When we pass our tensor to the output layer, the result will be the **prediction tensor**. Since our data has ten prediction classes, we know our output tensor will have ten elements.

```python
# (6) output layer
t = self.out(t)
#t = F.softmax(t, dim=1)
```
Inside the network we usually use `relu()` as our `non-linear activation function`, but for the output layer, whenever we have a single category that we are trying to predict, we use `softmax()`. The `softmax function` returns a positive probability for each of the prediction classes, and the probabilities sum to `1`.

**However**, in our case, we won't use `softmax()` because the loss function that we'll use, `F.cross_entropy()`, implicitly performs the `softmax()` operation on its input, so we'll just return the result of the last linear transformation.

The implication of this is that our network will be trained using the softmax operation but will not need to compute the additional operation when the network is used for inference after the training process is complete.

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size = 5)
        
        self.fc1 = nn.Linear(in_features = 12*4*4,out_features = 120)
        self.fc2 = nn.Linear(in_features = 120,out_features = 60)
        self.out = nn.Linear(in_features = 60,out_features = 10)
        
    def forward(self,t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)

        return t

## Quiz 05
Q1:In neural network programming, the forward method of a network instance explicitly defines the network's ______________<br>
A1:transformation

Q2:In neural network programming, the forward method of a network instance is the mapping that maps an input tensor to a prediction output tensor.<br>
A2:**True**

Q3:The input layer of any neural network is determined by the input data. For this reason, we can think of the input layer as the identity transformation. Mathematically, this is the function,$$f(x)=x$$
A3:**True**

Q4:In neural network programming, all layers that are not the input or output layers are called hidden layers.<br>
A4:**True**

Q5:In neural network programming, the operations that are defined using _______________ are called layers.<br>
A5:weights

Q6:In the most general sense, neural networks are mathematical functions. Terms like layers, activation functions, and weights, are just used to help describe the different parts.<br>
A6:**True**

# 06 